## Majority Vote Support Vector Classifier for emotion, based on 4 simple features

Binary classification for each emotion. Trained with k fold cross validation and a loop over multiple shufflings of the data. 
Balances data to account for imbalanced data (some emotions are not reported as often as others)

Now we do not take the mean or median over the features but we perform classification on all data and then do a majority vote.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import copy

In [39]:
# Data import

ZC = np.load('../preprocessing/zeroCrossings_frame100ms_hop50ms.npy')
SC = np.load('../preprocessing/spectralCentroid_frame100ms_hop50ms.npy')
SV = np.load('../preprocessing/spectralVariance_frame5000ms_hop2500ms.npy')
ST = np.load('../preprocessing/staticTempoLibrosa.npy')

labels = pd.read_csv('../preprocessing/labels.csv')

In [40]:
# Only keep the emotion labels 
# These correspond to the average number each specific emotion was reported

# For classifying we threshold these averages at 0.5. 
# Average > 0.5: this emotion corresponds to this song
# Average < 0.5: emotion does not belong to this song

labels = labels.drop(['genre',
             'var_amazement',
             'var_solemnity',
             'var_tenderness',
             'var_nostalgia',
             'var_calmness',
             'var_power',
             'var_joyful_activation',
             'var_tension',
             'var_sadness',
             'mood',
             'var_mood',
             'liked',
             'var_liked',
             'disliked',
             'var_disliked',
             'age',
             'var_age',
             'gender',
             'var_gender',
             'number_of_observations',
             'track id'             
            ], axis=1)

emotions = list(labels)

In [41]:
np.mean(labels)

amazement            0.134012
solemnity            0.195961
tenderness           0.176911
nostalgia            0.254279
calmness             0.299257
power                0.190244
joyful_activation    0.261896
tension              0.237685
sadness              0.185288
dtype: float64

------------------------------

In [42]:
# Make dataframe

Ndata_per_song = np.shape(ZC)[1]
assert Ndata_per_song != 400
[ZC, SC, SV, ST] = StrechArrays([ZC, SC, SV, ST])
assert np.shape(ZC) == np.shape(SC) == np.shape(SV) == np.shape(ST)

features_dict_medians = {'ZC': ZC.flatten(),
                         'SC': SC.flatten(),
                         'SV': SV.flatten(),
                         'StaticTempo': ST.flatten()}
features = pd.DataFrame(data=features_dict_medians)


# Remove NaN values
labels = pd.DataFrame(labels.loc[labels.index.repeat(Ndata_per_song)])  # repeat to meet the number of features
labels = labels.reset_index(drop=True)
features, labels = removeNaN(features, labels)
assert not features.isnull().any().any(), 'features contains NaN'

In [ ]:
# Doing the actual training of the support vector classifier over multple rounds of shuffeling
assert not features.isnull().any().any(), 'features contains NaN1'

threshold = 0.5

N_emotions = 9
N_features = 4

confusion_Matrix = np.zeros((N_emotions, 2, 2))

N_shuffles = 1
k = 2
accuracies = np.zeros((N_emotions, N_shuffles))

emotions = ['amazement']

assert not features.isnull().any().any(), 'features contains NaN2'

for e_idx, emotion in enumerate(emotions):
    assert not features.isnull().any().any(), 'features contains NaN3'
    
    for Siter in range(N_shuffles):
        assert not features.isnull().any().any(), 'features contains NaN4'
        
        thresholdedLabel = copy.deepcopy(labels[emotion])

        thresholdedLabel[thresholdedLabel >= threshold] = int(1)
        thresholdedLabel[thresholdedLabel < threshold] = int(0)
       
        assert not features.isnull().any().any(), 'features contains NaN5'
        
        features, thresholdedLabel = shuffleData(features, thresholdedLabel)
        assert not features.isnull().any().any(), 'features contains NaN6'
        confusion_Matrix[e_idx, :, :] += kFoldConfusionMatrix(k, features, thresholdedLabel)

        accuracies[e_idx, Siter] = np.sum(
            np.eye(2)*confusion_Matrix[e_idx, :, :])/np.sum(confusion_Matrix[e_idx, :, :])

In [ ]:
# Confusion matrix
for e_idx, emotion in enumerate(emotions):
    print(f'EMOTION {emotion}: \n')
    print(f'Accuracy: {np.mean(accuracies[e_idx, :])} +- {np.std(accuracies[e_idx, :])} \n')

    fig,ax = plt.subplots()
    CMATRIX = confusion_Matrix[e_idx, :, :]
    print(CMATRIX)
    print('\n\n')
    CM = ConfusionMatrixDisplay(CMATRIX)
    CM.plot(ax=ax)
    ax.set_title(f'{emotion}')

---------------------------

In [2]:
def StrechArrays(Listofarrays):
    """
    Since we possibly use different window and hop sizes for computing the simple features we have arrays of different size. 
    For majority vote classification we need data arrays of same length
    Since they all correspond to the same time axis, we can 'stretch' the array by repeating values
    This function returns 'stretched' arrays with lenght equal to the length of the largest array.
    """
    Nsongs = np.shape(Listofarrays[0])[0]
    maxlength = 0
    for array in Listofarrays:
        if not array.ndim == 1:
            maxlength = max(maxlength, np.shape(array)[1])
    SOL = []
    for array in Listofarrays:
        length = np.shape(array)[1] if not array.ndim == 1 else 1
        if not length == maxlength:
            newarray = np.zeros((Nsongs, maxlength))
            for song in range(Nsongs):
                arr = []
                for i in range(maxlength):
                    arr.append(array[song, int(i*length/maxlength)]
                               ) if not array.ndim == 1 else arr.append(array[song])
                newarray[song, :] = np.array(arr)
            assert np.shape(newarray) == (Nsongs, maxlength)
            SOL.append(newarray)
        else:
            assert np.shape(array) == (Nsongs, maxlength)
            SOL.append(array)
    return SOL

In [37]:
# Shuffle

def shuffleData(features, labels):
#     features = features.reset_index(drop=True)
#     labels = labels.reset_index(drop=True)
    assert not features.isnull().any().any(), 'features contains NaN111'
    n_data = len(features)
    idx = np.random.permutation(n_data)
    features, labels = features.reindex(idx), labels.reindex(idx)
    assert not features.isnull().any().any(), 'features contains NaN11'
    return features, labels

In [12]:
# K fold Crossvalidation

def kFoldConfusionMatrix(k, features, labels):
    
#     assert not features.isnull().any().any(), 'features contains NaN'
#     assert not labels.isnull().any().any(), 'labels contains NaN'

    kfold_train_metrics = []
    kfold_test_metrics = []

    confusion_Matrix_total = np.zeros((2, 2), dtype= 'int')

    total_input = features
    total_labels = labels

    cv = KFold(n_splits=k)

    for train_index, test_index in cv.split(total_input):        
        train_df, train_labels = total_input.iloc[train_index], total_labels.iloc[train_index]
        test_df, test_labels = total_input.iloc[test_index], total_labels.iloc[test_index]

        # Standardizing data
        mean = train_df.mean()
        std = train_df.std()

        train_df = (train_df - mean) / std
        test_df = (test_df - mean) / std

        # Classifier
        clf = svm.SVC(kernel='linear', class_weight='balanced')
        clf.fit(train_df, train_labels)

        kfold_train_metrics.append(clf.score(train_df, train_labels))
        kfold_test_metrics.append(clf.score(test_df, test_labels))

        assert np.shape(clf.predict(test_df)) == np.shape(test_labels)
        
        CM = confusion_matrix(test_labels, clf.predict(test_df), labels= [0,1])
        
        assert np.shape(CM) == (2,2)

        confusion_Matrix_total += CM
    return confusion_Matrix_total

In [5]:
def removeNaN(features, labels):
    # Look for NaN values in features dataframe and drop these rows and the corresponding rows in label dataframe

    NaNidx = np.where(features.isnull().any(axis=1).tolist())[0]

    F = features.drop(NaNidx, axis=0)
#     labels = pd.DataFrame(labels)
    L = labels.drop(NaNidx, axis=0)
    features = features.reset_index(drop=True)
    labels = labels.reset_index(drop=True)
    assert not (F.isnull()).any().any(), 'NaN values in features not removed'
    
    return F, L

In [14]:
np.where(features.isnull().any(axis=1).tolist())[0]

array([   357,   2913,   3965,   5108,   5984,   7926,   8535,   9917,
        10310,  12459,  19462,  19725,  21487,  23533,  26274,  26627,
        27495,  28371,  28380,  28601,  29338,  31120,  31985,  38040,
        40250,  44045,  45699,  46155,  49121,  49597,  49811,  51777,
        52260,  54099,  57106,  58851,  59265,  61372,  62968,  66739,
        66764,  68517,  69194,  69701,  70042,  73620,  75188,  77709,
        78872,  79076,  81245,  82214,  82599,  83052,  83793,  86684,
        87137,  88120,  88738,  89820,  89990,  92734,  98347,  98412,
       101314, 105960, 106556, 110006, 111254, 118891, 119033, 119610,
       121669, 123826, 125050, 125727, 126938, 128826, 132865, 135374,
       136891, 137190, 146884, 150876, 152019, 152653, 153471, 156349,
       157689, 159409, 159452, 160411, 160538, 162161, 163823, 166662,
       166909, 167007, 167161, 167499, 168755, 171698, 174136, 176506,
       178392, 179323, 180560, 184439, 185038, 189145, 191928, 199400,
      

In [ ]:
features.loc[[62275]]

In [ ]:
features